# Analyse de données de systèmes éducatifs

Nous travaillons sur 5 fichiers CSV qui contiennent des données sur les systèmes éducatifs de différents pays. 

Chaque fichier est considéré comme une table et l'ensemble des fichiers comme une base de données.

Nous allons donc commencer par essayer de comprendre les données qu'ils contiennent puis de charger les données et les explorer.

## Objectifs

1. Charger les données
2. Explorer les données
3. Nettoyer les données
4. Analyser les données
5. Visualiser les données
6. Interpréter les résultats
7. Conclusion et réponses aux problématiques

## Import des librairies nécessaires

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [46]:
# Fonction pour afficher les informations sur les données (récupérée d'un projet existant)
def infos (df):
    nb_li = df.shape[0]
    nb_co = df.shape[1]
    t = np.empty(nb_li)
    t.fill(nb_li)
    df_l_null = pd.DataFrame(df.T.isna().sum()) # tableau du nbe de nul par lignes (+800 000 lignes)
    df_c_null = pd.DataFrame(df.isna().sum()) # tableau du nbe de nul par colonnes (+65 colonnes)
    
    # nbe de lignes sans 'null'
    al = len([x for x in df_l_null[0] if x==0])
    nb_ss_null = pd.DataFrame([al]).rename(index={0:'lign_ss_null'}).T
    pct_ss_null = pd.DataFrame([al*100/nb_li]).rename(index={0:'lign_ss_null'}).T
    # nbe de lignes 'null'
    bl = len([x for x in df_l_null[0] if x==nb_co])
    nb_null = pd.DataFrame([bl]).rename(index={0:'lign_null'}).T
    pct_null = pd.DataFrame([bl*100/nb_li]).rename(index={0:'lign_null'}).T
    # nbe de lignes mixtes
    cl = len([x for x in df_l_null[0] if (x!=0 and x!=nb_co)])
    nb_mix = pd.DataFrame([cl]).rename(index={0:'lign_mix'}).T
    pct_mix = pd.DataFrame([cl*100/nb_li]).rename(index={0:'lign_mix'}).T
    infos_nb = pd.concat([nb_ss_null, nb_null, nb_mix],axis=1, sort=False).rename(index={0:'nb'})
    infos_pct = pd.concat([pct_ss_null, pct_null, pct_mix],axis=1, sort=False).rename(index={0:'pct'})
    infos_l = pd.concat([infos_nb,infos_pct], sort=False)
    # nbe de lignes total
    infos_l["lign_tot"] = [infos_l.T['nb'].sum(), infos_l.T['pct'].sum()]
    
    # nbe de colonnes sans 'null'
    ac = len([x for x in df_c_null[0] if x==0])
    nb_ss_null = pd.DataFrame([ac]).rename(index={0:'col_ss_null'}).T
    pct_ss_null = pd.DataFrame([ac*100/nb_co]).rename(index={0:'col_ss_null'}).T
    # nbe de colonnes 'null'
    bc = len([x for x in df_c_null[0] if x==nb_li])
    nb_null = pd.DataFrame([bc]).rename(index={0:'col_null'}).T
    pct_null = pd.DataFrame([bc*100/nb_co]).rename(index={0:'col_null'}).T
    # nbe de colonnes mixtes
    cc = len([x for x in df_c_null[0] if (x!=0 and x!=nb_li)])
    nb_mix = pd.DataFrame([cc]).rename(index={0:'col_mix'}).T
    pct_mix = pd.DataFrame([cc*100/nb_co]).rename(index={0:'col_mix'}).T
    infos_nb = pd.concat([nb_ss_null, nb_null, nb_mix],axis=1, sort=False).rename(index={0:'nb'})
    infos_pct = pd.concat([pct_ss_null, pct_null, pct_mix],axis=1, sort=False).rename(index={0:'pct'})
    infos_c = pd.concat([infos_nb,infos_pct], sort=False)
    # nbe de lignes total
    infos_c["col_tot"] = [infos_c.T['nb'].sum(), infos_c.T['pct'].sum()]
    
    infos = pd.concat([infos_l,infos_c], axis=1, sort=False)
    
    return infos

## Chargement des données

In [47]:
# Le dropna permet de supprimer les colonnes fantômes "Unamed" en fin de tableau
country_series = pd.read_csv('CSV/EdStatsCountry-Series.csv').dropna(how='all', axis='columns')
country = pd.read_csv('CSV/EdStatsCountry.csv').dropna(how='all', axis='columns')
data = pd.read_csv('CSV/EdStatsData.csv').dropna(how='all', axis='columns')
footnote = pd.read_csv('CSV/EdStatsFootNote.csv').dropna(how='all', axis='columns')
series = pd.read_csv('CSV/EdStatsSeries.csv').dropna(how='all', axis='columns')

# Création d'autres dataframes pour la partie nettoyage des données
country_series_c = country_series.copy()
country_c = country.copy()
data_c = data.copy()
footnote_c = footnote.copy()
series_c = series.copy()

## Exploration des colonnes des tables

### Table **EdStatsCountry-Series** :

Cette table représente les **séries d'indicateurs** pour chaque pays.

- **CountryCode** -> ISO 3166-1 alpha-3 : les codes d'identification ISO des différents pays du monde 
- Indication du **type de données** étudiées :
    - "**SP.POP.TOTL**" : Total population.
    - "**SP.POP.GROW**" : Population growth (annual %).
    - "**NY.GDP.PCAP.PP.CD**" : GDP (PIB) per capita, PPP (Parité de pouvoir d'achat) (current international $).
    - "**NY.GDP.MKTP.PP.KD**" : GDP, PPP (constant 2011 international $).
    - "**NY.GNP.MKTP.PP.CD**" : GNI (Revenu national brut), PPP (current international $).
    - "**NY.GDP.MKTP.PP.CD**" : GDP, PPP (current international $).
    - "**NY.GDP.PCAP.PP.KD**" : GDP per capita, PPP (constant 2011 international $).
    - "**NY.GNP.PCAP.PP.CD**" : GNI per capita, PPP (current international $).
    - "**SP.POP.1564.TO.ZS**" : Population ages 15-64 (% of total).
    - "**SP.POP.0014.TO.ZS**" : Population ages 0-14 (% of total).
    - "**.MA.ZS**" : Population, male (% of total).
    - "**.FE.ZS**" : Population, female (% of total).
    - "**NY.GNP.PCAP.CD**" : GNI per capita, Atlas method (current US$).
    - "**NY.GDP.PCAP.CD**" : GDP per capita (current US$).
    - "**NY.GDP.PCAP.KD**" : GDP per capita (constant 2010 US$).
    - "**SP.POP.1564.MA.IN**" : Population ages 15-64, male.
    - "**SP.POP.0014.TO**": Population ages 0-14, total.
    - "**SP.POP.1564.TO**" : Population ages 15-64, total.
    - "**SP.POP.1564.FE.IN**" : Population ages 15-64, female.
    - "**SP.POP.0014.MA.IN**" : Population ages 0-14, male.
    - "**SP.POP.0014.FE.IN**" : Population ages 0-14, female.
- **Description** et informations sur la source de données

### Table **EdStatsCountry** :

Cette table contient des **informations sur les pays**.

- **Country Code** -> ISO 3166-1 alpha-3 : les codes d'identification ISO des différents pays du monde
- **Short Name** : Nom court du pays
- **Table Name** : Nom du pays dans la table
- **Long Name** : Nom complet du pays
- **2-alpha code** : Code alpha-2 du pays
- **Currency Unit** : Unité monétaire du pays
- **Special Notes** : Notes spéciales sur le pays
- **Region** : Région du monde
- **Income Group** : Groupe de revenu
- **WB-2 code** : Code WB-2 du pays
- **National accounts base year** : Année de base des comptes nationaux
- **National accounts reference year** : Année de référence des comptes nationaux
- **SNA price valuation** : Evaluation des prix SNA (méthode d'évaluation des prix utilisée dans le cadre du système de comptabilité nationale)
- **Lending category** : Catégorie de prêt
- **Other groups** : Autres groupes
- **System of National Accounts** : Système de comptabilité nationale
- **Alternative conversion factor** : Facteur de conversion alternatif
- **PPP survey year** : Année de l'enquête sur la parité de pouvoir d'achat
- **Balance of Payments Manual in use** : Manuel de la balance des paiements en cours d'utilisation
- **External debt Reporting status** : Statut de déclaration de la dette extérieure
- **System of trad**e : Système de commerce
- **Government Accounting concep**t : Concept de comptabilité gouvernementale
- **IMF data dissemination standard** : Norme de diffusion des données du FMI
- **Latest population census** : Dernier recensement de la population
- **Latest household survey** : Dernière enquête auprès des ménages
- **Source of most recent Income and expenditure data** : Source des données les plus récentes sur les revenus et les dépenses
- **Vital registration complete** : Enregistrement vital complet
- **Latest agricultural census** : Dernier recensement agricole
- **Latest industrial data** : Dernières données industrielles
- **Latest trade data** : Dernières données commerciales
- **Latest water withdrawal data** : Dernières données sur les prélèvements d'eau

### Table **EdStatsData** :

Cette table contient les **données des indicateurs** pour chaque pays.

- **Country Name** : Nom du pays
- **Country Code** : Code du pays
- **Indicator Name** : Nom de l'indicateur
- **Indicator Code** : Code de l'indicateur
- **1960 - 2015** : Valeurs de l'indicateur pour chaque année
- **2016 - 2100** : Prévisions de l'indicateur pour chaque année

### Table **EdStatsFootNote** :

Cette table contient des **notes de bas de page** pour les indicateurs.

- **CountryCode** : Code du pays
- **SeriesCode** : Code de la série
- **Year** : Année
- **Description** : Description de la note de bas de page

### Table **EdStatsSeries** :

Cette table contient des **informations sur les séries d'indicateurs**.

- **Series Code** : Code de la série
- **Topic** : Thème
- **Indicator Name** : Nom de l'indicateur
- **Short definition** : Définition courte
- **Long definition** : Définition longue
- **Unit of measure** : Unité de mesure
- **Periodicity** : Périodicité
- **Base Period** : Période de base
- **Other notes** : Autres notes
- **Aggregation method** : Méthode d'agrégation
- **Limitations and exceptions** : Limitations et exceptions
- **Notes from original source** : Notes de la source originale
- **General comments** : Commentaires généraux
- **Source** : Source
- **Statistical concept and methodology** : Concept statistique et méthodologie
- **Development relevance** : Pertinence du développement
- **Related source links** : Liens de source connexes
- **Other web links** : Autres liens web
- **Related indicators** : Indicateurs connexes
- **License Type** : Type de licence

## Exploration des données

### Analyse des valeurs manquantes ou nulles dans les tables

In [48]:
## Nbre de null par ligne et par colonne
infos_t = pd.concat([infos(data), infos(country), infos(series),\
                  infos(country_series), infos(footnote)], axis = 0,\
                 keys=['data', 'country', 'series', 'cnt_ser', 'footnote'])
pd.options.display.float_format = '{:.1f}'.format
infos_t

lign_ss_null  lign_null  lign_mix  lign_tot  col_ss_null  \
data     nb            0.0        0.0  886930.0  886930.0          4.0   
         pct           0.0        0.0     100.0     100.0          5.8   
country  nb            0.0        0.0     241.0     241.0          4.0   
         pct           0.0        0.0     100.0     100.0         12.9   
series   nb            0.0        0.0    3665.0    3665.0          5.0   
         pct           0.0        0.0     100.0     100.0         33.3   
cnt_ser  nb          613.0        0.0       0.0     613.0          3.0   
         pct         100.0        0.0       0.0     100.0        100.0   
footnote nb       643638.0        0.0       0.0  643638.0          4.0   
         pct         100.0        0.0       0.0     100.0        100.0   

              col_null  col_mix  col_tot  
data     nb        0.0     65.0     69.0  
         pct       0.0     94.2    100.0  
country  nb        0.0     27.0     31.0  
         pct       0.0     87.1    100.0  
series   nb        0.0     10.0     15.0  
         pct       0.0     66.7    100.0  
cnt_ser  nb        0.0      0.0      3.0  
         pct       0.0      0.0    100.0  
footnote nb        0.0      0.0      4.0  
         pct       0.0      0.0    100.0

### Analyse par table

#### Table **EdStatsCountry-Series** :

Affichage des **5 premières lignes** de la table

In [49]:
country_series.head()

,CountryCode,SeriesCode,DESCRIPTION
0,ABW,SP.POP.TOTL,Data sources : United Nations World Population...
1,ABW,SP.POP.GROW,Data sources: United Nations World Population ...
2,AFG,SP.POP.GROW,Data sources: United Nations World Population ...
3,AFG,NY.GDP.PCAP.PP.CD,Estimates are based on regression.
4,AFG,SP.POP.TOTL,Data sources : United Nations World Population...


**Description** des colonnes de la table

In [50]:
country_series.describe()

,CountryCode,SeriesCode,DESCRIPTION
count,613,613,613
unique,211,21,97
top,MDA,SP.POP.TOTL,Data sources : United Nations World Population...
freq,18,211,154


**Informations** sur les colonnes de la table

In [51]:
country_series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CountryCode  613 non-null    object
 1   SeriesCode   613 non-null    object
 2   DESCRIPTION  613 non-null    object
dtypes: object(3)
memory usage: 14.5+ KB


Comptage des **valeurs manquantes ou nulles** dans les colonnes de la table

In [52]:
country_series.isnull().sum()

CountryCode    0
SeriesCode     0
DESCRIPTION    0
dtype: int64

Calcul du nombre de **valeurs dupliquées** dans la table

In [53]:
country_series.duplicated().sum()

0

#### Table **EdStatsCountry** :

Affichage des **5 premières lignes** de la table

In [54]:
country.head()

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,SNA data for 2000-2011 are updated from offici...,Latin America & Caribbean,High income: nonOECD,AW,...,NaN,NaN,2010,NaN,NaN,Yes,NaN,NaN,2012.0,NaN
1,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",NaN,2013/14,NaN,2012.0,2000
2,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,"April 2013 database update: Based on IMF data,...",Sub-Saharan Africa,Upper middle income,AO,...,Budgetary central government,General Data Dissemination System (GDDS),1970,"Malaria Indicator Survey (MIS), 2011","Integrated household survey (IHS), 2008",NaN,2015,NaN,NaN,2005
3,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,NaN,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2010.0,2012.0,2006
4,AND,Andorra,Andorra,Principality of Andorra,AD,Euro,NaN,Europe & Central Asia,High income: nonOECD,AD,...,NaN,NaN,2011. Population figures compiled from adminis...,NaN,NaN,Yes,NaN,NaN,2006.0,NaN


**Description** des colonnes de la table

In [55]:
country.describe()

,National accounts reference year,Latest industrial data,Latest trade data
count,32.0,107.0,185.0
mean,2001.5,2008.1,2011.0
std,5.2,2.6,2.6
min,1987.0,2000.0,1995.0
25%,1996.8,2007.5,2011.0
50%,2002.0,2009.0,2012.0
75%,2005.0,2010.0,2012.0
max,2012.0,2010.0,2012.0


**Informations** sur les colonnes de la table

In [56]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 31 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Country Code                                       241 non-null    object 
 1   Short Name                                         241 non-null    object 
 2   Table Name                                         241 non-null    object 
 3   Long Name                                          241 non-null    object 
 4   2-alpha code                                       238 non-null    object 
 5   Currency Unit                                      215 non-null    object 
 6   Special Notes                                      145 non-null    object 
 7   Region                                             214 non-null    object 
 8   Income Group                                       214 non-null    object 
 9   WB-2 code 

Comptage des **valeurs manquantes ou nulles** dans les colonnes de la table

In [57]:
country.isnull().sum()

Country Code                                           0
Short Name                                             0
Table Name                                             0
Long Name                                              0
2-alpha code                                           3
Currency Unit                                         26
Special Notes                                         96
Region                                                27
Income Group                                          27
WB-2 code                                              1
National accounts base year                           36
National accounts reference year                     209
SNA price valuation                                   44
Lending category                                      97
Other groups                                         183
System of National Accounts                           26
Alternative conversion factor                        194
PPP survey year                

Calcul du nombre de **valeurs dupliquées** dans la table

In [58]:
country.duplicated().sum()

0

#### Table **EdStatsData** :

Affichage des **5 premières lignes** de la table

In [59]:
data.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
0,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arab World,ARB,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,54.8,54.9,56.2,57.3,58.0,59.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Description** des colonnes de la table

In [60]:
data.describe()

,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
count,72288.0,35537.0,35619.0,35545.0,35730.0,87306.0,37483.0,37574.0,37576.0,36809.0,...,51436.0,51436.0,51436.0,51436.0,51436.0,51436.0,51436.0,51436.0,51436.0,51436.0
mean,1974772449.6,4253638470.2,4592364905.3,5105005610.9,5401492903.4,2314287511.6,5731808142.1,6124437154.2,6671488719.0,7436723955.0,...,714.6,722.5,727.1,728.4,726.6,722.8,717.7,711.3,703.4,694.0
std,121168685351.9,180481446225.2,191408271479.8,205917043980.7,211214985371.1,137505922744.7,221554619977.1,232548901067.4,247398632281.3,266095745106.0,...,21368.5,22158.4,22879.9,23523.4,24081.5,24559.0,24965.9,25301.8,25560.7,25741.9
min,-1.4,-1.6,-3.1,-4.0,-4.2,-3.7,-3.0,-3.2,-3.6,-3.0,...,-1.8,-1.6,-1.4,-1.3,-1.1,-0.9,-0.8,-0.7,-0.6,-0.5
25%,0.9,8.9,9.2,9.6,9.9,1.4,9.3,9.5,10.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,6.3,63.2,66.6,69.7,70.9,9.7,71.0,71.3,72.9,75.1,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
75%,62.5,56552.0,58636.5,62029.0,63836.8,78.5,56828.0,57391.8,59404.2,64115.0,...,7.5,7.5,7.5,7.3,7.1,6.7,6.1,5.5,4.7,4.0
max,19039286948196.1,19864566419298.1,21009161433401.4,22383671023495.4,22829911729819.1,23006343161670.4,24241276811572.6,25213826643314.4,26221014860337.7,27308727186242.9,...,2813669.8,2951568.8,3070878.8,3169710.6,3246239.2,3301586.2,3337871.2,3354746.3,3351886.9,3330483.5


**Informations** sur les colonnes de la table

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886930 entries, 0 to 886929
Data columns (total 69 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country Name    886930 non-null  object 
 1   Country Code    886930 non-null  object 
 2   Indicator Name  886930 non-null  object 
 3   Indicator Code  886930 non-null  object 
 4   1970            72288 non-null   float64
 5   1971            35537 non-null   float64
 6   1972            35619 non-null   float64
 7   1973            35545 non-null   float64
 8   1974            35730 non-null   float64
 9   1975            87306 non-null   float64
 10  1976            37483 non-null   float64
 11  1977            37574 non-null   float64
 12  1978            37576 non-null   float64
 13  1979            36809 non-null   float64
 14  1980            89122 non-null   float64
 15  1981            38777 non-null   float64
 16  1982            37511 non-null   float64
 17  1983      

Comptage des **valeurs manquantes ou nulles** dans les colonnes de la table

In [62]:
data.isnull().sum()

Country Name           0
Country Code           0
Indicator Name         0
Indicator Code         0
1970              814642
                   ...  
2080              835494
2085              835494
2090              835494
2095              835494
2100              835494
Length: 69, dtype: int64

Calcul du nombre de **valeurs dupliquées** dans la table

In [63]:
data.duplicated().sum()

0

#### Table **EdStatsFootNote** :

Affichage des **5 premières lignes** de la table

In [64]:
footnote.head()

,CountryCode,SeriesCode,Year,DESCRIPTION
0,ABW,SE.PRE.ENRL.FE,YR2001,Country estimation.
1,ABW,SE.TER.TCHR.FE,YR2005,Country estimation.
2,ABW,SE.PRE.TCHR.FE,YR2000,Country estimation.
3,ABW,SE.SEC.ENRL.GC,YR2004,Country estimation.
4,ABW,SE.PRE.TCHR,YR2006,Country estimation.


**Description** des colonnes de la table

In [65]:
footnote.describe()

,CountryCode,SeriesCode,Year,DESCRIPTION
count,643638,643638,643638,643638
unique,239,1558,56,9102
top,LIC,SH.DYN.MORT,YR2004,Country Data
freq,7320,9226,27128,191188


**Informations** sur les colonnes de la table

In [66]:
footnote.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643638 entries, 0 to 643637
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   CountryCode  643638 non-null  object
 1   SeriesCode   643638 non-null  object
 2   Year         643638 non-null  object
 3   DESCRIPTION  643638 non-null  object
dtypes: object(4)
memory usage: 19.6+ MB


Comptage des **valeurs manquantes ou nulles** dans les colonnes de la table

In [67]:
footnote.isnull().sum()

CountryCode    0
SeriesCode     0
Year           0
DESCRIPTION    0
dtype: int64

Calcul du nombre de **valeurs dupliquées** dans la table

In [68]:
footnote.duplicated().sum()

0

#### Table **EdStatsSeries** :

Affichage des **5 premières lignes** de la table

In [69]:
series.head()

,Series Code,Topic,Indicator Name,Short definition,Long definition,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,General comments,Source,Statistical concept and methodology,Development relevance,Related source links
0,BAR.NOED.1519.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 15-19 with...,Percentage of female population age 15-19 with...,NaN,NaN,NaN,NaN,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN
1,BAR.NOED.1519.ZS,Attainment,Barro-Lee: Percentage of population age 15-19 ...,Percentage of population age 15-19 with no edu...,Percentage of population age 15-19 with no edu...,NaN,NaN,NaN,NaN,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN
2,BAR.NOED.15UP.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 15+ with n...,Percentage of female population age 15+ with n...,NaN,NaN,NaN,NaN,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN
3,BAR.NOED.15UP.ZS,Attainment,Barro-Lee: Percentage of population age 15+ wi...,Percentage of population age 15+ with no educa...,Percentage of population age 15+ with no educa...,NaN,NaN,NaN,NaN,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN
4,BAR.NOED.2024.FE.ZS,Attainment,Barro-Lee: Percentage of female population age...,Percentage of female population age 20-24 with...,Percentage of female population age 20-24 with...,NaN,NaN,NaN,NaN,NaN,NaN,Robert J. Barro and Jong-Wha Lee: http://www.b...,NaN,NaN,NaN


**Description** des colonnes de la table

In [70]:
series.describe()

,Series Code,Topic,Indicator Name,Short definition,Long definition,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,General comments,Source,Statistical concept and methodology,Development relevance,Related source links
count,3665,3665,3665,2156,3665,99,314,552,47,14,14,3665,23,3,215
unique,3665,37,3665,1169,2060,1,4,14,3,9,8,31,2,1,1
top,BAR.NOED.1519.FE.ZS,Learning Outcomes,Barro-Lee: Percentage of female population age...,Data Interpretation: 1=Latent; 2=Emerging; 3=E...,Data Interpretation: 1=Latent; 2=Emerging; 3=E...,Annual,Projections (2010 to 2100),EGRA,Weighted average,Data should be used cautiously because of diff...,When NEET rates are available for more than tw...,UNESCO Institute for Statistics,TIMSS,Unemployment and total employment are the broa...,http://saber.worldbank.org/index.cfm
freq,1,1046,1,215,215,99,308,403,31,3,3,1269,20,3,215


**Informations** sur les colonnes de la table

In [71]:
series.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Series Code                          3665 non-null   object
 1   Topic                                3665 non-null   object
 2   Indicator Name                       3665 non-null   object
 3   Short definition                     2156 non-null   object
 4   Long definition                      3665 non-null   object
 5   Periodicity                          99 non-null     object
 6   Base Period                          314 non-null    object
 7   Other notes                          552 non-null    object
 8   Aggregation method                   47 non-null     object
 9   Limitations and exceptions           14 non-null     object
 10  General comments                     14 non-null     object
 11  Source                               3665 n

Comptage des **valeurs manquantes ou nulles** dans les colonnes de la table

In [72]:
series.isnull().sum()

Series Code                               0
Topic                                     0
Indicator Name                            0
Short definition                       1509
Long definition                           0
Periodicity                            3566
Base Period                            3351
Other notes                            3113
Aggregation method                     3618
Limitations and exceptions             3651
General comments                       3651
Source                                    0
Statistical concept and methodology    3642
Development relevance                  3662
Related source links                   3450
dtype: int64

Calcul du nombre de **valeurs dupliquées** dans la table

In [73]:
series.duplicated().sum()

0